# Characterize predicted mutations

This notebook processes the ranked mutations in `paper/mutations.tsv` and generates output plots for examining mutational enrichment.

In [ ]:
# imports
library(data.table)
library(tidyverse)
library(testit)
library(pheatmap)

In [ ]:
# Set graphics
options(repr.plot.width=10, repr.plot.height=10)
output_plot_dir <- 'paper/mutation_scoring/'
plot_dpi = 300

In [ ]:
# Read input data
res_mutations <- fread('paper/mutations.tsv')

In [ ]:
# Helper functions
strpart <- function(x, split, n, fixed=FALSE) {
    sapply(strsplit(as.character(x),split,fixed=fixed),'[',n)
}

matrix_to_long_df <- function(matrix) {
    rn <- rownames(matrix)
    cn <- colnames(matrix)
    df <- as.data.frame(matrix)
    colnames(df) <- cn
    df$rownames <- rn
    pivot_longer(df, !rownames, names_to = "colnames")
}

## Get summary of mutations at the top of the list

In [ ]:
get_mutation_summary <- function(mutations, n_mutations = NULL, return_matrix = TRUE, select_gene = NULL) {
    aa_chars = sort(c('G','A','L','M','F','W','K','Q','E','S','P','V','I','C','Y','H','R','N','D','T'))
    
    if (!is.null(n_mutations)) {
        mutations[1:n_mutations,] %>% pull('mutation') -> mutation_strings
    } else {
        mutations %>% pull('mutation') -> mutation_strings
    }
    
    # Extract mutation information
    gene <- strpart(mutation_strings, ':', 1)
    aa_change <- strpart(mutation_strings, ':', 2)
    aa_from <- substr(aa_change,1,1)
    position <- substr(aa_change,2,nchar(aa_change)-1)
    aa_to <- substr(aa_change,nchar(aa_change),nchar(aa_change))
    mutation_data_parse <- data.frame(
        gene,
        aa_from,
        position,
        aa_to
    )
    
    # Subset to specific gene
    if (!is.null(select_gene)) {
        mutation_data_parse <- mutation_data_parse[gene == select_gene,]
    }
    
    # Filter to only amino acid changes (otherwise it includes 'e')
    mutation_data_parse %>% 
        filter(aa_from %in% aa_chars) %>%
        filter(aa_to %in% aa_chars) -> mutation_data_parse
    
    # Count mutations
    mutation_data_parse[,c('aa_from','aa_to')] %>% 
        group_by(aa_from, aa_to) %>% summarise(n=n(), .groups="drop") -> summarized_counts
    

    if (return_matrix) {
        # pivot wider
        summarized_counts %>% pivot_wider(
            id_cols = c(aa_from, aa_to),
            names_from = aa_to,
            values_from = n,
            values_fill=0
        ) -> mutation_counts_actual
        
        # Turn to matrix
        rn <- mutation_counts_actual$aa_from
        cn <- colnames(mutation_counts_actual)
        mutation_counts_actual <- (as.matrix(mutation_counts_actual[,-1]))
        rownames(mutation_counts_actual) <- rn
        colnames(mutation_counts_actual) <- cn[-1]

        # Find missing cols and rows
        missing_cols <- setdiff(aa_chars, colnames(mutation_counts_actual))
        missing_rows <- setdiff(aa_chars, rownames(mutation_counts_actual))

        # Add missing cols
        original_cn <- colnames(mutation_counts_actual)
        mutation_counts_actual <- cbind(mutation_counts_actual, 
                                        matrix(0, nrow(mutation_counts_actual), 
                                               length(missing_cols)))
        colnames(mutation_counts_actual) <- c(original_cn, missing_cols)
        # Add missing rows
        original_rn <- rownames(mutation_counts_actual)
        mutation_counts_actual <- rbind(mutation_counts_actual,
                                        matrix(0, length(missing_rows),
                                              ncol(mutation_counts_actual)))
        rownames(mutation_counts_actual) <- c(original_rn, missing_rows)

        # Put in order
        mutation_counts_actual <- mutation_counts_actual[aa_chars,aa_chars]
        
        # Rows -- from
        # Cols -- to
        mutation_counts_actual <- t(mutation_counts_actual)

        return(mutation_counts_actual)
    } else {
        return(summarized_counts)
    }

}

In [ ]:
# Mutation type occurences in top 1000 entries
n_mutations <- 1000

In [ ]:
# For confirming orientation of output
get_mutation_summary(res_mutations, n_mutations = n_mutations, return_matrix=FALSE) %>% 
    filter(aa_from == 'I' & aa_to == 'T')

In [ ]:
matrix_to_long_df(get_mutation_summary(res_mutations, n_mutations = n_mutations)) %>% 
    ggplot(aes(x=rownames, y=colnames, fill=value)) + 
    geom_tile() +
    theme_bw() +
    theme(axis.text.x = element_text(color='black', size=14), axis.text.y = element_text(color='black', size=14), axis.title.x = element_text(color='black', size=14), axis.title.y = element_text(color='black', size=14),
         legend.position = "none") +
    scale_fill_gradient(low = "white", high = "red") +
    geom_text(aes(label=value)) +
    ggtitle(paste0('Mutation Frequencies in Top ', n_mutations, ' mutations')) +
    scale_x_discrete(name='To AA') +
    scale_y_discrete(name='From AA') -> plot_aa_frequencies

plot_aa_frequencies

In [ ]:
ggsave(paste0(output_plot_dir, 'top_substitutions.png'), width=6, height=6, dpi = plot_dpi)

## Test for enrichment of particular mutations

In [ ]:
test_mutation_enrichment <- function(res_mutations, n_mutations=1000, symmetric = TRUE, select_gene = NULL) {
    aa_chars = sort(c('G','A','L','M','F','W','K','Q','E','S','P','V','I','C','Y','H','R','N','D','T'))
    
    top_mut <- get_mutation_summary(res_mutations, n_mutations, select_gene = select_gene)
    universe_mut <- get_mutation_summary(res_mutations, NULL, select_gene = select_gene)
    
    pvals <- matrix(0, length(aa_chars), length(aa_chars))
    rownames(pvals) <- colnames(pvals) <- aa_chars
    
    top_total <- sum(top_mut)
    universe_total <- sum(universe_mut)
    
    universe_rate <- universe_mut / universe_total

    for (from_aa in aa_chars) {
        for (to_aa in aa_chars) {
            if (from_aa == to_aa) {
                # Do not test synonymous
                pvals[from_aa, to_aa] <- NA
            } else {
                if (symmetric) {
                    this_mutation_count = top_mut[from_aa, to_aa] + top_mut[to_aa, from_aa] # observed count
                    this_mutation_global_prob = universe_rate[from_aa, to_aa] + universe_rate[to_aa, from_aa] #expected_rate
                } else {
                    this_mutation_count = top_mut[from_aa, to_aa] # observed count
                    this_mutation_global_prob = universe_rate[from_aa, to_aa] # excpected rate
                }
                pvals[from_aa, to_aa] <- dbinom(this_mutation_count, top_total, this_mutation_global_prob)
            }
        }
    }
    
    pvals    
}

In [ ]:
# Confirm symmetric output when symmetric comparison
assert(isSymmetric(test_mutation_enrichment(res_mutations, 100, symmetric=TRUE)))

In [ ]:
# Look for maximal mutational enrichment while scanning for number of genes considered
summary_fn_min <- function(x) {min(x, na.rm = TRUE)}
summary_fn_mean <- function(x) {mean(x, na.rm = TRUE)}

test_mutation_enrichment_serial <- function(res_mutations, n_start=1, n_end=1000, step=1, symmetric=TRUE, summary_fn=summary_fn_min, select_gene = NULL) {
    xs <- seq(n_start, n_end, step)
    ys <- unlist(lapply(xs, function(x) {
        summary_fn(test_mutation_enrichment(res_mutations,n=x, symmetric=symmetric, select_gene = select_gene))
    }))
    most_sign_pval <- data.frame(n=xs, most_sign_pval= ys)
    most_sign_pval
}

In [ ]:
# Look both with synonymous and non-synonymour changes considered
# Get mean and maximal p-value for each
n_end <- 3000

most_sign_pval_sym <- test_mutation_enrichment_serial(res_mutations, n_end=n_end, step=10, symmetric = TRUE, summary_fn=summary_fn_min)
most_sign_pval_sym$type <- 'most'
most_sign_pval_sym$sym <- 'symmetric'

most_sign_pval_nonsym <- test_mutation_enrichment_serial(res_mutations, n_end=n_end, step=10,symmetric = FALSE, summary_fn=summary_fn_min)
most_sign_pval_nonsym$type <- 'most'
most_sign_pval_nonsym$sym <- 'non-symmetric'

mean_sign_pval_sym <- test_mutation_enrichment_serial(res_mutations, n_end=n_end, step=10,symmetric = TRUE, summary_fn=summary_fn_mean)
mean_sign_pval_sym$type <- 'mean'
mean_sign_pval_sym$sym <- 'symmetric'

mean_sign_pval_nonsym <- test_mutation_enrichment_serial(res_mutations, n_end=n_end, step=10,symmetric = FALSE, summary_fn=summary_fn_mean)
mean_sign_pval_nonsym$type <- 'mean'
mean_sign_pval_nonsym$sym <- 'non-symmetric'

plot_data <- rbind(most_sign_pval_sym, most_sign_pval_nonsym, mean_sign_pval_sym, mean_sign_pval_nonsym)

In [ ]:
# multiple testing corrected p-value for all tests except diagonal
# we can be less stringent in the synonymous case
pval_cutoff <- 0.05 / (20*20-20)

In [ ]:
dim(res_mutations)

In [ ]:
ggplot(plot_data, aes(x=n, y=-log10(most_sign_pval), color=type, linetype=sym)) + 
    geom_line() +
    geom_hline(yintercept = -log10(pval_cutoff)) + 
    geom_hline(yintercept = -log10(pval_cutoff*2), linetype=2) +
    facet_wrap(~sym) +
    theme_bw() + 
    theme(axis.text.x = element_text(color='black', size=14), axis.text.y = element_text(color='black', size=16), axis.title.x = element_text(color='black', size=16), axis.title.y = element_text(color='black', size=14),
         legend.position = "none") +
    scale_x_continuous(name="n top genes") + 
    scale_y_continuous(name="-log10(min(pval))") -> plot_pval_vs_top_genes

plot_pval_vs_top_genes

In [ ]:
ggsave(paste0(output_plot_dir, 'pvals_vs_top_genes.png'), width=6, height=6, dpi = plot_dpi)

In [ ]:
test_individual_mutation_enrichment_serial <- function(res_mutations, n_start=1, n_end=1000, step=1, symmetric=TRUE, select_gene = NULL) {
    ns <- seq(n_start, n_end, step)
    info <- do.call(rbind, lapply(ns, function(x) {
        df.tmp <- matrix_to_long_df(test_mutation_enrichment(res_mutations,n=x,symmetric=symmetric, select_gene = select_gene))
        df.tmp$n <- x
        df.tmp$aa_change = paste0(df.tmp$rownames,'->', df.tmp$colnames)
        df.tmp$rownames = NULL
        df.tmp$colnames = NULL
        df.tmp
    }))
    info
}

In [ ]:
info <- test_individual_mutation_enrichment_serial(res_mutations,1,1000, symmetric=FALSE)

In [ ]:
info %>%
    ggplot(aes(x=n,y=-log10(value), color = aa_change)) + 
    geom_line() + 
    theme_bw() + 
    ggtitle('Individual Mutation Significance') +
    theme(legend.position='none') + 
    scale_x_continuous(name='n top mutations') + 
    scale_y_continuous(name='-log10(p-value)') -> plot_individual_mutation_significance

plot_individual_mutation_significance

In [ ]:
ggsave(paste0(output_plot_dir,'plot_individual_mutation_significance.png'), width=6, height=6, dpi = plot_dpi)

In [ ]:
info %>% filter(!is.na(value)) %>% 
    group_by(aa_change) %>% 
    summarise(min_pval = min(value)) %>% 
    arrange(min_pval) %>% 
    filter(min_pval < pval_cutoff)

In [ ]:
info %>% filter(!is.na(value)) %>% 
    filter(aa_change == 'K->N') %>%
    ggplot(aes(x=n,y=-log10(value), color = aa_change)) + 
    geom_line() + 
    theme_bw() + 
    ggtitle('Lys -> Asn (K->N) Mutation Significance') +
    theme(legend.position='none') + 
    scale_x_continuous(name='n top mutations') + 
    scale_y_continuous(name='-log10(p-value)') -> plot_n_to_k_mutation_significance

plot_n_to_k_mutation_significance

In [ ]:
ggsave(paste0(output_plot_dir,'plot_k_to_n_mutation_significance.png'), width=6, height=6, dpi = plot_dpi)

## Tabulate Top 1000 Mutations by Gene

In [ ]:
n_mutations <- 1000

unique(strpart(res_mutations$mutation,':',1)) -> sars_cov2_genes
names(sars_cov2_genes) <- sars_cov2_genes

do.call(rbind, lapply(sars_cov2_genes, function(g) {
    get_mutation_summary(res_mutations, n_mutations = n_mutations, select_gene = g,return_matrix = FALSE) %>% 
        mutate(aa_change=paste0(aa_from, '->', aa_to)) %>%
        mutate(aa_from = NULL, aa_to = NULL, gene = g) -> df
    df
})) -> df

df %>% pivot_wider(id_cols = c(aa_change, gene), values_from = n, names_from = aa_change,values_fill = 0) -> tmp1
tmp_rn <- tmp1$gene
tmp1 <- tmp1[,-1]
tmp1 <- as.matrix(tmp1)
rownames(tmp1) <- tmp_rn
n_mutations_by_gene <- tmp1

In [ ]:
options(repr.plot.width=10, repr.plot.height=30)
pheatmap(t(tmp1),display_numbers = TRUE, number_format = '%.0f', fontsize_number = 12)
options(repr.plot.width=10, repr.plot.height=10)

In [ ]:
# repeat saving to disk
pheatmap(t(tmp1),display_numbers = TRUE, number_format = '%.0f', fontsize_number = 12, filename = paste0(output_plot_dir,'by_gene_heatmap.png'), width=10, height=30)

# Mutational Enrichment on a per-gene basis

In [ ]:
unique(strpart(res_mutations$mutation, ':', 1)) -> available_genes
available_genes

## S gene

In [ ]:
get_serial_enrichment_by_gene <- function(res_mutations, select_gene, n_end = 3000,  step=10) {
    most_sign_pval_sym <- test_mutation_enrichment_serial(res_mutations, n_end=n_end, step=10, symmetric = TRUE, summary_fn=summary_fn_min, select_gene = select_gene)
    most_sign_pval_sym$type <- 'most'
    most_sign_pval_sym$sym <- 'symmetric'

    most_sign_pval_nonsym <- test_mutation_enrichment_serial(res_mutations, n_end=n_end, step=10,symmetric = FALSE, summary_fn=summary_fn_min, select_gene = select_gene)
    most_sign_pval_nonsym$type <- 'most'
    most_sign_pval_nonsym$sym <- 'non-symmetric'

    mean_sign_pval_sym <- test_mutation_enrichment_serial(res_mutations, n_end=n_end, step=10,symmetric = TRUE, summary_fn=summary_fn_mean, select_gene = select_gene)
    mean_sign_pval_sym$type <- 'mean'
    mean_sign_pval_sym$sym <- 'symmetric'

    mean_sign_pval_nonsym <- test_mutation_enrichment_serial(res_mutations, n_end=n_end, step=10,symmetric = FALSE, summary_fn=summary_fn_mean, select_gene = select_gene)
    mean_sign_pval_nonsym$type <- 'mean'
    mean_sign_pval_nonsym$sym <- 'non-symmetric'

    rbind(most_sign_pval_sym, most_sign_pval_nonsym, mean_sign_pval_sym, mean_sign_pval_nonsym)   
}

In [ ]:
get_serial_enrichment_by_gene(res_mutations, select_gene = 'S') -> serial_enrichment_S

In [ ]:
serial_enrichment_S %>% 
    ggplot(aes(x=n, y=-log10(most_sign_pval), color=type, linetype=sym)) + 
        geom_line() +
        geom_hline(yintercept = -log10(pval_cutoff)) + 
        geom_hline(yintercept = -log10(pval_cutoff*2), linetype=2) +
        facet_wrap(~sym) +
        theme_bw() + 
        theme(axis.text.x = element_text(color='black', size=14), axis.text.y = element_text(color='black', size=16), axis.title.x = element_text(color='black', size=16), axis.title.y = element_text(color='black', size=14),
         legend.position = "none") +
        scale_x_continuous(name="n top genes") + 
        scale_y_continuous(name="-log10(min(pval))") +
        ggtitle('Enrichment in Spike') -> plot_pval_vs_top_genes_S

plot_pval_vs_top_genes_S

In [ ]:
ggsave(paste0(output_plot_dir,'plot_pval_vs_top_genes_S.png'), width=6, height=6, dpi = plot_dpi, plot = plot_pval_vs_top_genes_S)

Look at per aa

In [ ]:
info <- test_individual_mutation_enrichment_serial(res_mutations,1,1000, symmetric=FALSE, select_gene = 'S')

In [ ]:
info %>%
    ggplot(aes(x=n,y=-log10(value), color = aa_change)) + 
    geom_line() + 
    theme_bw() + 
    ggtitle('Individual Mutation Significance') +
    theme(legend.position='none') + 
    scale_x_continuous(name='n top mutations') + 
    scale_y_continuous(name='-log10(p-value)') -> plot_individual_mutation_significance_S

plot_individual_mutation_significance_S

In [ ]:
ggsave(paste0(output_plot_dir,'plot_individual_mutation_significance_S.png'), width=6, height=6, dpi = plot_dpi, plot = plot_individual_mutation_significance_S)

In [ ]:
info %>% filter(!is.na(value)) %>% 
    group_by(aa_change) %>% 
    summarise(min_pval = min(value)) %>% 
    arrange(min_pval) %>% 
    filter(min_pval < pval_cutoff)

In [ ]:
info %>% filter(!is.na(value)) %>% 
    filter(aa_change == 'K->N') %>%
    ggplot(aes(x=n,y=-log10(value), color = aa_change)) + 
    geom_line() + 
    theme_bw() + 
    theme(axis.text.x = element_text(color='black', size=14), axis.text.y = element_text(color='black', size=16), axis.title.x = element_text(color='black', size=16), axis.title.y = element_text(color='black', size=14), legend.position = "none") +
    ggtitle('Lys -> Asn (K->N) Mutation Significance') +
    theme(legend.position='none') + 
    scale_x_continuous(name='n top mutations') + 
    scale_y_continuous(name='-log10(p-value)') -> plot_n_to_k_mutation_significance_S__K_to_N

plot_n_to_k_mutation_significance_S__K_to_N

In [ ]:
ggsave(paste0(output_plot_dir,'plot_individual_mutation_significance_S__K_to_N.png'), width=6, height=6, dpi = plot_dpi, plot = plot_n_to_k_mutation_significance_S__K_to_N)

## M gene

In [ ]:
get_serial_enrichment_by_gene(res_mutations, select_gene = 'M') -> serial_enrichment_M

In [ ]:
serial_enrichment_M %>% 
    ggplot(aes(x=n, y=-log10(most_sign_pval), color=type, linetype=sym)) + 
        geom_line() +
        geom_hline(yintercept = -log10(pval_cutoff)) + 
        geom_hline(yintercept = -log10(pval_cutoff*2), linetype=2) +
        theme(axis.text.x = element_text(color='black', size=14), axis.text.y = element_text(color='black', size=16), axis.title.x = element_text(color='black', size=16), axis.title.y = element_text(color='black', size=14), legend.position = "none") +
        facet_wrap(~sym) +
        theme_bw() + scale_x_continuous(name="n top genes") + 
        scale_y_continuous(name="-log10(min(pval))") +
        ggtitle('Enrichment in M') -> plot_pval_vs_top_genes_M

plot_pval_vs_top_genes_M

In [ ]:
ggsave(paste0(output_plot_dir,'plot_individual_mutation_significance_M.png'), width=6, height=6, dpi = plot_dpi, plot = plot_pval_vs_top_genes_M)

## N gene

In [ ]:
get_serial_enrichment_by_gene(res_mutations, select_gene = 'N') -> serial_enrichment_N

In [ ]:
serial_enrichment_N %>% 
    ggplot(aes(x=n, y=-log10(most_sign_pval), color=type, linetype=sym)) + 
        geom_line() +
        theme_bw() +
        geom_hline(yintercept = -log10(pval_cutoff)) + 
        geom_hline(yintercept = -log10(pval_cutoff*2), linetype=2) +
        theme(axis.text.x = element_text(color='black', size=14), axis.text.y = element_text(color='black', size=16), axis.title.x = element_text(color='black', size=16), axis.title.y = element_text(color='black', size=14), legend.position = "none") +
        facet_wrap(~sym) +
         scale_x_continuous(name="n top genes") + 
        scale_y_continuous(name="-log10(min(pval))") +
        ggtitle('Enrichment in N') -> plot_pval_vs_top_genes_N

plot_pval_vs_top_genes_N

In [ ]:
ggsave(paste0(output_plot_dir,'plot_pval_vs_top_genes_N.png'), width=6, height=6, dpi = plot_dpi, plot = plot_pval_vs_top_genes_N)

## ORF9b

In [ ]:
get_serial_enrichment_by_gene(res_mutations, select_gene = 'ORF9b') -> serial_enrichment_ORF9b

In [ ]:
serial_enrichment_ORF9b %>% 
    ggplot(aes(x=n, y=-log10(most_sign_pval), color=type, linetype=sym)) + 
        geom_line() +
        geom_hline(yintercept = -log10(pval_cutoff)) + 
        geom_hline(yintercept = -log10(pval_cutoff*2), linetype=2) +
        facet_wrap(~sym) +
        theme_bw() + scale_x_continuous(name="n top genes") + 
        scale_y_continuous(name="-log10(min(pval))") +
        ggtitle('Enrichment in ORF9b') -> plot_pval_vs_top_genes_ORF9b

plot_pval_vs_top_genes_ORF9b

In [ ]:
ggsave(paste0(output_plot_dir,'plot_pval_vs_top_genes_ORF9b.png'), width=6, height=6, dpi = plot_dpi, plot = plot_pval_vs_top_genes_ORF9b)

## ORF1a

In [ ]:
get_serial_enrichment_by_gene(res_mutations, select_gene = 'ORF1a') -> serial_enrichment_ORF1a

In [ ]:
serial_enrichment_ORF1a %>% 
    ggplot(aes(x=n, y=-log10(most_sign_pval), color=type, linetype=sym)) + 
        geom_line() +
        geom_hline(yintercept = -log10(pval_cutoff)) + 
        geom_hline(yintercept = -log10(pval_cutoff*2), linetype=2) +
        facet_wrap(~sym) +
        theme_bw() + scale_x_continuous(name="n top genes") + 
        scale_y_continuous(name="-log10(min(pval))") +
        ggtitle('Enrichment in ORF1a') -> plot_pval_vs_top_genes_ORF1a

plot_pval_vs_top_genes_ORF1a

In [ ]:
ggsave(paste0(output_plot_dir,'plot_pval_vs_top_genes_ORF1a.png'), width=6, height=6, dpi = plot_dpi, plot = plot_pval_vs_top_genes_ORF1a)

## ORF1b

In [ ]:
get_serial_enrichment_by_gene(res_mutations, select_gene = 'ORF1b') -> serial_enrichment_ORF1b

In [ ]:
serial_enrichment_ORF1b %>% 
    ggplot(aes(x=n, y=-log10(most_sign_pval), color=type, linetype=sym)) + 
        geom_line() +
        geom_hline(yintercept = -log10(pval_cutoff)) + 
        geom_hline(yintercept = -log10(pval_cutoff*2), linetype=2) +
        facet_wrap(~sym) +
        theme_bw() + scale_x_continuous(name="n top genes") + 
        scale_y_continuous(name="-log10(min(pval))") +
        ggtitle('Enrichment in ORF1b') -> plot_pval_vs_top_genes_ORF1b

plot_pval_vs_top_genes_ORF1b

In [ ]:
ggsave(paste0(output_plot_dir,'plot_pval_vs_top_genes_ORF1b.png'), width=6, height=6, dpi = plot_dpi, plot = plot_pval_vs_top_genes_ORF1b)

## ORF3a

In [ ]:
get_serial_enrichment_by_gene(res_mutations, select_gene = 'ORF3a') -> serial_enrichment_ORF3a

In [ ]:
serial_enrichment_ORF3a %>% 
    ggplot(aes(x=n, y=-log10(most_sign_pval), color=type, linetype=sym)) + 
        geom_line() +
        geom_hline(yintercept = -log10(pval_cutoff)) + 
        geom_hline(yintercept = -log10(pval_cutoff*2), linetype=2) +
        facet_wrap(~sym) +
        theme_bw() + scale_x_continuous(name="n top genes") + 
        scale_y_continuous(name="-log10(min(pval))") +
        ggtitle('Enrichment in ORF3a') -> plot_pval_vs_top_genes_ORF3a

plot_pval_vs_top_genes_ORF3a

In [ ]:
ggsave(paste0(output_plot_dir,'plot_pval_vs_top_genes_ORF3a.png'), width=6, height=6, dpi = plot_dpi, plot = plot_pval_vs_top_genes_ORF3a)

## E

In [ ]:
get_serial_enrichment_by_gene(res_mutations, select_gene = 'E') -> serial_enrichment_E

In [ ]:
serial_enrichment_E %>% 
    ggplot(aes(x=n, y=-log10(most_sign_pval), color=type, linetype=sym)) + 
        geom_line() +
        geom_hline(yintercept = -log10(pval_cutoff)) + 
        geom_hline(yintercept = -log10(pval_cutoff*2), linetype=2) +
        facet_wrap(~sym) +
        theme_bw() + scale_x_continuous(name="n top genes") + 
        scale_y_continuous(name="-log10(min(pval))") +
        ggtitle('Enrichment in E') -> plot_pval_vs_top_genes_E

plot_pval_vs_top_genes_E

In [ ]:
ggsave(paste0(output_plot_dir,'plot_pval_vs_top_genes_E.png'), width=6, height=6, dpi = plot_dpi, plot = plot_pval_vs_top_genes_E)

# ORF7b

In [ ]:
get_serial_enrichment_by_gene(res_mutations, select_gene = 'ORF7b') -> serial_enrichment_ORF7b

In [ ]:
serial_enrichment_ORF7b %>% 
    ggplot(aes(x=n, y=-log10(most_sign_pval), color=type, linetype=sym)) + 
        geom_line() +
        geom_hline(yintercept = -log10(pval_cutoff)) + 
        geom_hline(yintercept = -log10(pval_cutoff*2), linetype=2) +
        facet_wrap(~sym) +
        theme_bw() + scale_x_continuous(name="n top genes") + 
        scale_y_continuous(name="-log10(min(pval))") +
        ggtitle('Enrichment in ORF7b') -> plot_pval_vs_top_genes

plot_pval_vs_top_genes_ORF7b

In [ ]:
ggsave(paste0(output_plot_dir,'plot_pval_vs_top_genes_ORF7b.png'), width=6, height=6, dpi = plot_dpi, plot = plot_pval_vs_top_genes_ORF7b)

# ORF8

In [ ]:
get_serial_enrichment_by_gene(res_mutations, select_gene = 'ORF8') -> serial_enrichment_ORF8

In [ ]:
serial_enrichment_ORF8 %>% 
    ggplot(aes(x=n, y=-log10(most_sign_pval), color=type, linetype=sym)) + 
        geom_line() +
        geom_hline(yintercept = -log10(pval_cutoff)) + 
        geom_hline(yintercept = -log10(pval_cutoff*2), linetype=2) +
        facet_wrap(~sym) +
        theme_bw() + scale_x_continuous(name="n top genes") + 
        scale_y_continuous(name="-log10(min(pval))") +
        ggtitle('Enrichment in ORF8') -> plot_pval_vs_top_genes_ORF8

plot_pval_vs_top_genes_ORF8

In [ ]:
ggsave(paste0(output_plot_dir,'plot_pval_vs_top_genes_ORF8.png'), width=6, height=6, dpi = plot_dpi, plot = plot_pval_vs_top_genes_ORF8)

# ORF6

In [ ]:
get_serial_enrichment_by_gene(res_mutations, select_gene = 'ORF6') -> serial_enrichment_ORF6

In [ ]:
serial_enrichment_ORF6 %>% 
    ggplot(aes(x=n, y=-log10(most_sign_pval), color=type, linetype=sym)) + 
        geom_line() +
        geom_hline(yintercept = -log10(pval_cutoff)) + 
        geom_hline(yintercept = -log10(pval_cutoff*2), linetype=2) +
        facet_wrap(~sym) +
        theme_bw() + scale_x_continuous(name="n top genes") + 
        scale_y_continuous(name="-log10(min(pval))") +
        ggtitle('Enrichment in ORF6') -> plot_pval_vs_top_genes_ORF6

plot_pval_vs_top_genes_ORF6

In [ ]:
ggsave(paste0(output_plot_dir,'plot_pval_vs_top_genes_ORF6.png'), width=6, height=6, dpi = plot_dpi, plot = plot_pval_vs_top_genes_ORF6)

# ORF10

In [ ]:
get_serial_enrichment_by_gene(res_mutations, select_gene = 'ORF10') -> serial_enrichment_ORF10

In [ ]:
serial_enrichment_ORF10 %>% 
    ggplot(aes(x=n, y=-log10(most_sign_pval), color=type, linetype=sym)) + 
        geom_line() +
        geom_hline(yintercept = -log10(pval_cutoff)) + 
        geom_hline(yintercept = -log10(pval_cutoff*2), linetype=2) +
        facet_wrap(~sym) +
        theme_bw() + scale_x_continuous(name="n top genes") + 
        scale_y_continuous(name="-log10(min(pval))") +
        ggtitle('Enrichment in ORF10') -> plot_pval_vs_top_genes_ORF10

plot_pval_vs_top_genes_ORF10

In [ ]:
ggsave(paste0(output_plot_dir,'plot_pval_vs_top_genes_ORF10.png'), width=6, height=6, dpi = plot_dpi, plot = plot_pval_vs_top_genes_ORF10)